### Постановка задачи

Просмотр фильмов на оригинальном языке - это популярный и действенный метод обучения иностранным языкам. Важно выбрать фильм, который подходит студенту по уровню сложности, т.е. студент понимал бы 50-70 % диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и решить, какому уровню он соответствует. Однако это требует больших временных затрат.

Наша задача - разработать ML решение для автоматического определения уровня сложности англоязычных фильмов. 

Программа-максимум:

           - языковая модель, 
           - веб-интерфейс,
           - микросервис.  
           
### Знакомство с исходными данными

В качестве исходных данных нам предоставлены размеченные по уровню сложности фильмы и субтитры к ним, словари английского языка с набором слов по уровню.

Сначала взглянем на эти файлы, чтоб понять, как можно было бы с ними вообще работать.

In [1]:
import pysrt
import warnings
import pandas as pd
import os
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

#nltk.download('stopwords')
#nltk.download('wordnet')
warnings.filterwarnings('ignore')
RANDOM = 123

In [2]:
path_A2 = './English_level/English scores/Subtitles_all/A2/The Walking Dead-S01E01-Days Gone Bye.English.srt'

In [3]:
subs = pysrt.open(path_A2)

In [4]:
len(subs)

673

In [5]:
for i in range(0, len(subs), 50):
    print(subs[i])

1
00:00:03,169 --> 00:00:05,171
( bugs chittering )

51
00:06:24,376 --> 00:06:27,045
- And what do you say to that?
- I know what I want to say.

101
00:09:19,207 --> 00:09:22,085 X1:0
Dispatcher:
Unit 1, unit 3, to eastbound Route 18,

151
00:12:53,996 --> 00:12:56,332
Shh shh. That's it. Do you hear me?
Shh shh shh. Okay.

201
00:20:23,438 --> 00:20:25,440
( bugs chittering )

251
00:28:20,153 --> 00:28:23,030
Take a moment,
look how sharp it is.

301
00:31:39,304 --> 00:31:40,388
Yeah.

351
00:34:22,709 --> 00:34:26,086
Man: Don't look.
Get away from the windows.

401
00:38:37,536 --> 00:38:39,705
That's right.

451
00:41:46,000 --> 00:41:47,835
Conserve your ammo.

501
00:47:57,069 --> 00:47:59,071
( growling )

551
00:52:00,594 --> 00:52:02,429
You're all right.
Go on. You're all right.

601
00:57:15,775 --> 00:57:17,485
Good boy.

651
01:05:33,926 --> 01:05:36,595
♪ Of the place I know so well ♪



Чтобы проанализировать слова, нужно из этой структуры убрать тайминг, нумерацию кадров (видимо, это строки, потому что их количество равно длине файла с субтитрами), убрать слова в скобках, потому что в задаче запрос на понимание диалогов, а в скобках не диалоги, убрать знаки препинания и цифры, привести к нижнему регистру.

### Создание единого датасета из предложенных данных

In [6]:

basepath = './English_level/English scores/Subtitles_all/'

df_1 = pd.DataFrame(columns = ['Level', 'Movie', 'Srt'])


for s in ('A2', 'B1', 'B2', 'C1'):
    
    path = os.path.join(basepath, s)
    
    for file in os.listdir(path):
        
        with open(os.path.join(path, file), 'r', encoding='ISO-8859-1') as infile:
            txt = infile.read()
            
        df_1.loc[len(df_1)] = [s, file[:-4], txt]
        


In [7]:
display(df_1)

,Level,Movie,Srt
0,A2,The Walking Dead-S01E01-Days Gone Bye.English,"1\n00:00:03,169 --> 00:00:05,171\n( bugs chitt..."
1,A2,The Walking Dead-S01E02-Guts.English,"1\n00:00:03,045 --> 00:00:05,047\n- ( birds ch..."
2,A2,The Walking Dead-S01E03-Tell It To The Frogs.E...,"1\n00:00:03,003 --> 00:00:04,671\n( thunder ru..."
3,A2,The Walking Dead-S01E04-Vatos.English,"1\n00:00:03,045 --> 00:00:05,422\n( birds chir..."
4,A2,The Walking Dead-S01E05-Wildfire.English,"1\n00:00:03,420 --> 00:00:04,922\n- ( walkie-t..."
...,...,...,...
158,C1,Suits.S03E06.720p.HDTV.x264-mSD,"ï»¿1\n00:00:01,383 --> 00:00:02,751\nI lost Av..."
159,C1,Suits.S03E07.HDTV.x264-mSD,"ï»¿1\n00:00:00,052 --> 00:00:01,352\nPreviousl..."
160,C1,Suits.S03E08.480p.HDTV.x264-mSD,"ï»¿1\n00:00:01,436 --> 00:00:03,028\nI get Ava..."
161,C1,Suits.S03E09.480p.HDTV.x264-mSD,"ï»¿1\n00:00:00,024 --> 00:00:01,478\nPreviousl..."


В первой таблице, собранной из данных папки заказчика с распределением субтитров по уровню языка,  у нас есть полный набор данных - название фильма, его рейтинг, текст субтитров.

In [8]:

df_2 = pd.DataFrame(columns = ['Movie', 'Srt'])
path = os.path.join(basepath, 'Subtitles')
for file in os.listdir(path):
    if file!='.DS_Store':
        
        with open(os.path.join(path, file), 'r', encoding='ISO-8859-1') as infile:
            txt = infile.read()
        df_2.loc[len(df_2)] = [file[:-4], txt]
        


In [9]:
display(df_2)

,Movie,Srt
0,10_Cloverfield_lane(2016),"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,10_things_I_hate_about_you(1999),"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,Aladdin(1992),"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
3,All_dogs_go_to_heaven(1989),"1\n00:00:12,319 --> 00:00:14,821\nCAPTIONING M..."
4,An_American_tail(1986),"ï»¿1\n00:02:24,080 --> 00:02:26,528\n(INDISTIN..."
...,...,...
110,Warm_bodies(2013),"2\n00:00:26,559 --> 00:00:28,627\n<i>What am I..."
111,Westworld_scenes_of_Dr_Robert_Ford,"1\n00:00:00,000 --> 00:00:21,179\n[Music]\n\n2..."
112,We_are_the_Millers(2013),"1\n00:00:02,400 --> 00:00:03,731\n<i>Oh, my Go..."
113,While_You_Were_Sleeping(1995),"1\n00:02:20,760 --> 00:02:24,720\nLUCY: <i>Oka..."


Во второй таблице, тоже собранной из данных файлов в папке заказчика, но другой, не размеченной по уровню языка, есть названия фильмов и текст субтитров.

In [10]:
df_3 = pd.read_excel('./English_level/English scores/movies_labels.xlsx')

In [11]:
display(df_3)

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+
...,...,...,...
236,236,Matilda(2022),C1
237,237,Bullet train,B1
238,238,Thor: love and thunder,B2
239,239,Lightyear,B2


In [12]:
df_3.drop(columns='id', axis=1)

,Movie,Level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+
...,...,...
236,Matilda(2022),C1
237,Bullet train,B1
238,Thor: love and thunder,B2
239,Lightyear,B2


В третьей таблице, предоставленной самим заказчиком, есть названия фильмов и уровень языка.

Теперь нам предстоит объединить эти данные в одну таблицу.

Столбцом для объединения точно будет название фильма. Объединение df_3 с df_2 не составит труда.

In [13]:
df_2_3 = df_3.merge(df_2, on='Movie', how='outer').drop(columns='id')

In [14]:
len(df_2_3)

250

In [15]:
len(df_2)

115

In [16]:
len(df_3)

241

In [17]:
df_2_3.isna().sum()

Movie      0
Level      9
Srt      131
dtype: int64

In [18]:
df_2_3.sample(10)

,Movie,Level,Srt
124,Seven.Worlds.One.Planet.S01E01.2160p.BluRay.Re...,B1,NaN
102,The_usual_suspects(1995),B2,"1\n00:03:34,463 --> 00:03:36,328\nHow you doin..."
152,Ghosts.of.Girlfriends.Past.2009.BluRay.720p.x2...,B2,NaN
244,Gogo_Loves_English,NaN,"1\n00:00:01,130 --> 00:00:18,359\n[Music]\n\n2..."
197,Suits.S02E14.HDTV.x264-ASAP,B2,NaN
137,Crazy4TV.com - Suits.S06E02.720p.BluRay.x265.H...,B2,NaN
229,Suits.S03E05.480p.HDTV.x264-mSD,C1,NaN
232,Suits.S03E08.480p.HDTV.x264-mSD,C1,NaN
141,Crazy4TV.com - Suits.S06E06.720p.BluRay.x265.H...,B2,NaN
185,Suits.S02E02.HDTV.x264-ASAP,B2,NaN


In [19]:
df_all = df_2_3.merge(df_1, on='Movie', how='outer')

In [20]:
df_all.sample(4)

,Movie,Level_x,Srt_x,Level_y,Srt_y
131,Spirit.Stallion.of.the.Cimarron.EN,B1,NaN,B1,"1\n00:00:36,883 --> 00:00:46,554\n''Spirit Sta..."
150,Crazy4TV.com - Suits.S06E15.720p.BluRay.x265.H...,B2,NaN,B2,"1\n00:00:02,252 --> 00:00:04,346\n<font color=..."
246,Pride_and_Prejudice,NaN,"1\n00:00:20,020 --> 00:00:42,009\n[Music]\n\n2...",NaN,NaN
175,Suits.S01E04.1080p.BluRay.AAC5.1.x265-DTG.02.EN,B2,NaN,B2,"ï»¿1\n00:00:04,754 --> 00:00:07,339\nHarvard T..."


In [21]:
df_all.isna().sum()

Movie        0
Level_x     49
Srt_x      171
Level_y    127
Srt_y      127
dtype: int64

In [22]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Movie    290 non-null    object
 1   Level_x  241 non-null    object
 2   Srt_x    119 non-null    object
 3   Level_y  163 non-null    object
 4   Srt_y    163 non-null    object
dtypes: object(5)
memory usage: 11.5+ KB


In [23]:
df_all['Level_x'].unique()

array(['B1', 'B2', 'A2/A2+', 'C1', 'B1, B2', 'A2/A2+, B1', 'A2', nan],
      dtype=object)

In [24]:
df_all['Level_y'].unique()

array([nan, 'A2', 'B1', 'B2', 'C1'], dtype=object)

Иногда указывают несколько уровней языка для одного фильма (А2, В1). Пока я оставлю самый высокий, это проще. Можно отложить эти сомнительные фильмы и потом определить обученной моделью их уровень. Уровень А2/А2+ заменяем на А2, потому что у нас не стоит задачи в градации подуровней внутри уровня.

In [25]:
df_all['Level_x'] = df_all['Level_x'].replace('A2/A2+', 'A2')
df_all['Level_x'] = df_all['Level_x'].replace('A2/A2+, B1', 'B1')

In [26]:
df_all['Level_x'] = df_all['Level_x'].replace('B1, B2', 'B2')

In [27]:
df_all['Level_x'].unique()

array(['B1', 'B2', 'A2', 'C1', nan], dtype=object)

In [28]:
print('Строк с разными значениями в столбцах Label_x и Label_y:',\
    len(df_all.loc[(df_all['Level_x'].notna())&(df_all['Level_y'].notna()) & (df_all['Level_x']!=df_all['Level_y'])]))

Строк с разными значениями в столбцах Label_x и Label_y: 0


Значит, просто заполним пропуски переносом значений из одного стобца в другой.

In [29]:
df_all['Level']=df_all['Level_x'].fillna(df_all['Level_y'])

In [30]:
df_all['Level'].unique()

array(['B1', 'B2', 'A2', 'C1', nan], dtype=object)

In [31]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Movie    290 non-null    object
 1   Level_x  241 non-null    object
 2   Srt_x    119 non-null    object
 3   Level_y  163 non-null    object
 4   Srt_y    163 non-null    object
 5   Level    281 non-null    object
dtypes: object(6)
memory usage: 13.7+ KB


Также проверим, есть ли несовпадения по субтитрам.

In [32]:
print('Строк с разными значениями в столбцах Srt_x и Srt_y:',\
    len(df_all.loc[(df_all['Srt_x'].notna())&(df_all['Srt_y'].notna()) & (df_all['Srt_x']!=df_all['Srt_y'])]))

Строк с разными значениями в столбцах Srt_x и Srt_y: 0


In [33]:
df_all['Srt']=df_all['Srt_x'].fillna(df_all['Srt_y'])

In [34]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Movie    290 non-null    object
 1   Level_x  241 non-null    object
 2   Srt_x    119 non-null    object
 3   Level_y  163 non-null    object
 4   Srt_y    163 non-null    object
 5   Level    281 non-null    object
 6   Srt      282 non-null    object
dtypes: object(7)
memory usage: 16.0+ KB


In [35]:
df_all.columns

Index(['Movie', 'Level_x', 'Srt_x', 'Level_y', 'Srt_y', 'Level', 'Srt'], dtype='object')

In [36]:
df = df_all.copy().drop(columns=['Level_x', 'Srt_x', 'Level_y', 'Srt_y'])

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   290 non-null    object
 1   Level   281 non-null    object
 2   Srt     282 non-null    object
dtypes: object(3)
memory usage: 6.9+ KB


Если пропуски в разных строках, то удалится в общей сложности 17 строк. Пока восстанавливать пропущенные значения не буду - надо смочь выполнить работу в минимальном объёме.

In [38]:
df_grade = df.loc[(df['Level'].isna())&(df['Srt'].notna())]

Отделяю фильмы с субтитрами, но неопределённым уровнем языка.

In [39]:
df_grade

,Movie,Level,Srt
241,Breaking_Bad_The_Movie(2017),NaN,"1\n00:00:05,299 --> 00:00:08,220\nwhat hi good..."
242,Bren╨Т.Brown.The.Call.to.Courage.2019.720.NF.7...,NaN,"ï»¿1\n00:00:07,216 --> 00:00:09,676\n[presente..."
243,Casper,NaN,"1\n00:00:18,040 --> 00:00:21,870\n[Music]\n\n2..."
244,Gogo_Loves_English,NaN,"1\n00:00:01,130 --> 00:00:18,359\n[Music]\n\n2..."
245,Harry_Potter_and_the_philosophers_stone(2001),NaN,"1\n00:01:22,065 --> 00:01:27,070\nI should've ..."
246,Pride_and_Prejudice,NaN,"1\n00:00:20,020 --> 00:00:42,009\n[Music]\n\n2..."
247,The_Ghost_Writer,NaN,"1\n00:00:04,670 --> 00:00:18,579\n[Music]\n\n2..."
248,Up(2009),NaN,"1\n00:00:46,040 --> 00:00:50,960\n{\i1}Movieto..."
249,Westworld_scenes_of_Dr_Robert_Ford,NaN,"1\n00:00:00,000 --> 00:00:21,179\n[Music]\n\n2..."


In [40]:
df = df.dropna(subset=['Level', 'Srt'])

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 273 entries, 0 to 289
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Movie   273 non-null    object
 1   Level   273 non-null    object
 2   Srt     273 non-null    object
dtypes: object(3)
memory usage: 8.5+ KB


Теперь посмотрим на дубли. Найдём - удалим.

In [42]:
df.duplicated().sum()

3

In [43]:
df.drop_duplicates()

df = df.reset_index()

df.head(8)

,index,Movie,Level,Srt
0,0,10_Cloverfield_lane(2016),B1,"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,1,10_things_I_hate_about_you(1999),B1,"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,2,A_knights_tale(2001),B2,"1\n00:00:15,089 --> 00:00:21,229\nResync: Xenz..."
3,3,A_star_is_born(2018),B2,"1\n00:00:17,610 --> 00:00:22,610\n- <i><font c..."
4,4,Aladdin(1992),A2,"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
5,5,All_dogs_go_to_heaven(1989),A2,"1\n00:00:12,319 --> 00:00:14,821\nCAPTIONING M..."
6,6,An_American_tail(1986),A2,"ï»¿1\n00:02:24,080 --> 00:02:26,528\n(INDISTIN..."
7,7,Babe(1995),A2,"0\n00:02:31,114 --> 00:02:34,316\nThis is a ta..."


In [44]:
df.drop('index', axis=1)

,Movie,Level,Srt
0,10_Cloverfield_lane(2016),B1,"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,10_things_I_hate_about_you(1999),B1,"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,A_knights_tale(2001),B2,"1\n00:00:15,089 --> 00:00:21,229\nResync: Xenz..."
3,A_star_is_born(2018),B2,"1\n00:00:17,610 --> 00:00:22,610\n- <i><font c..."
4,Aladdin(1992),A2,"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
...,...,...,...
268,Virgin.River.S01E06.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:18,852 --> 00:00:19,852\nHey.\n\n2\n0..."
269,Virgin.River.S01E07.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:10,468 --> 00:00:13,178\nAre you sure..."
270,Virgin.River.S01E08.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:07,382 --> 00:00:10,012\nTwo IVs in p..."
271,Virgin.River.S01E09.INTERNAL.720p.WEB.x264-STRiFE,B2,"1\n00:00:16,474 --> 00:00:18,024\nOh.\n\n2\n00..."


In [45]:
del df_1, df_2, df_3, df_2_3, df_all

Удалила уже ненужные переменные.

### Обработка субтитров

Необходимо преобразовать текст в числовую форму, удобную для моделирования.

Сначала избавимся от лишних символов, стоп-слов и скобок.

In [71]:
stop = stopwords.words('english') + ["i'm", "i'll", "oh", "hey"]

In [76]:
stop


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

Нужно оставить только текст, поэтому с помощью регулярных выражений удалим ненужное - небуквенные символы, содержимое скобок, html-теги, переносы, пробелы больше одного подряд.

In [47]:
del_n = re.compile('\n')                  # перенос каретки
del_tags = re.compile('<[^>]*>')          # html-теги
del_brackets = re.compile('\([^)]*\)')    # содержимое круглых скобок
clean_text = re.compile('[^а-яa-z\s\']')  # все небуквенные символы кроме пробелов и апострофов
del_spaces = re.compile('\s{2,}')         

def prepare_text(text):
    text = del_n.sub(' ', str(text).lower())
    text = del_tags.sub('', text)
    text = del_brackets.sub('', text)
    res_text = clean_text.sub('', text)
    res_text = res_text.lower()  #перевод в нижний регистр
    return del_spaces.sub(' ',res_text)
    


In [48]:
df.head(8)

,index,Movie,Level,Srt
0,0,10_Cloverfield_lane(2016),B1,"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,1,10_things_I_hate_about_you(1999),B1,"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,2,A_knights_tale(2001),B2,"1\n00:00:15,089 --> 00:00:21,229\nResync: Xenz..."
3,3,A_star_is_born(2018),B2,"1\n00:00:17,610 --> 00:00:22,610\n- <i><font c..."
4,4,Aladdin(1992),A2,"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
5,5,All_dogs_go_to_heaven(1989),A2,"1\n00:00:12,319 --> 00:00:14,821\nCAPTIONING M..."
6,6,An_American_tail(1986),A2,"ï»¿1\n00:02:24,080 --> 00:02:26,528\n(INDISTIN..."
7,7,Babe(1995),A2,"0\n00:02:31,114 --> 00:02:34,316\nThis is a ta..."


Следующая функция разбивает текст субтитров на слова и удаляет те, которые встречаются в списке стоп-слов.
Пока не очень понимаю, пригодится ли длина первоначального текста, но я её посчитаю. И длину образовавшегося массива слов.

In [49]:
def df_stop_words(Sr, stop):

    for itm in range(len(Sr)):
        line = Sr[itm]
        line = prepare_text(line)

        line = [w for w in line.split() if w not in stop]
        Sr[itm] = line

    return Sr



In [74]:
df['Srt'] = df_stop_words(df['Srt'], stop)

In [75]:
df['Srt']

0      ['fix', 'sync', 'bozxphd', 'enjoy', 'flick', '...
1      ['hey', 'right', 'cameron', 'go', 'nine', 'sch...
2      ['resync', 'xenzainef', 'retail', 'help', he',...
3      ['sync', 'correct', 'mrcjnthn', 'get', 'black'...
4      ['oh', 'come', 'land', 'faraway', 'place', 'ca...
                             ...                        
268    ['hey', 'hey', 'chainsaw', 'easier', 'yeah', '...
269    ['sure', can't, 'convinc', 'stay', 'right', 't...
270    ['two', 'iv', 'place', 'antecubit', 'fossa', '...
271    ['oh', 'oh', 'yeah', 'okay', 'hey', 'hey', 'ri...
272    ['long', 'known', i'v, 'suspect', 'coupl', 'we...
Name: Srt, Length: 273, dtype: object

In [52]:
df.head(8)

,index,Movie,Level,Srt
0,0,10_Cloverfield_lane(2016),B1,"[fixed, synced, bozxphd, enjoy, flick, ben, ph..."
1,1,10_things_I_hate_about_you(1999),B1,"[hey, right, cameron, go, nine, schools, years..."
2,2,A_knights_tale(2001),B2,"[resync, xenzainef, retail, help, he's, due, l..."
3,3,A_star_is_born(2018),B2,"[synced, corrected, mrcjnthn, get, black, eyes..."
4,4,Aladdin(1992),A2,"[oh, come, land, faraway, place, caravan, came..."
5,5,All_dogs_go_to_heaven(1989),A2,"[captioning, made, possible, mgm, home, entert..."
6,6,An_American_tail(1986),A2,"[mama, tanya, fievel, stop, twirling, twirling..."
7,7,Babe(1995),A2,"[tale, aboutan, unprejudiced, heart, changed, ..."


Теперь проведём лемматизацию - приведём слова к корневой форме.

In [53]:
#stemmer = WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()


In [54]:
def lemmatise_srt(Sr):
    for itm in range(len(Sr)):
        line = Sr[itm]
        line = [porter.stem(word) for word in line]
        Sr[itm] =  str(line)
    return Sr
    

In [55]:
df['Srt'] = lemmatise_srt(df['Srt'])

In [56]:
df.head(8)

,index,Movie,Level,Srt
0,0,10_Cloverfield_lane(2016),B1,"['fix', 'sync', 'bozxphd', 'enjoy', 'flick', '..."
1,1,10_things_I_hate_about_you(1999),B1,"['hey', 'right', 'cameron', 'go', 'nine', 'sch..."
2,2,A_knights_tale(2001),B2,"['resync', 'xenzainef', 'retail', 'help', ""he'..."
3,3,A_star_is_born(2018),B2,"['sync', 'correct', 'mrcjnthn', 'get', 'black'..."
4,4,Aladdin(1992),A2,"['oh', 'come', 'land', 'faraway', 'place', 'ca..."
5,5,All_dogs_go_to_heaven(1989),A2,"['caption', 'made', 'possibl', 'mgm', 'home', ..."
6,6,An_American_tail(1986),A2,"['mama', 'tanya', 'fievel', 'stop', 'twirl', '..."
7,7,Babe(1995),A2,"['tale', 'aboutan', 'unprejud', 'heart', 'chan..."


In [57]:
df.drop(columns=['index']).to_csv('movie_data.csv', index=False)

In [58]:
#df = pd.read_csv('movie_data.csv')  
#df.head(8)


Теперь приступим к векторизации.

Может быть так, что в предыдущих обработках какие-то слова испортились или в тексте встречаются опечатки, поэтому в качестве параметра векторизации возьму минимальное число документов, в которых встречается слово, - два.

In [60]:
count = CountVectorizer(min_df=2) #vocabulary=Mapping - рассмотреть этот параметр

bag = count.fit_transform(df['Srt'])

In [61]:
tfidfconverter = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
bag = tfidfconverter.fit_transform(bag).toarray()

Преобразование из слов в цифры закончилось. Теперь эту матрицу надо соотнести с таргетом и построить на их основе модель предсказания.

### Построение модели

#### Обучающая и тестовая выборки

Я собираюсь использовать кросс-валидацию, поэтому валидационная выборка  не нужна, только тестовая и обучающая.

Разделю в соотношении 70/30.

In [62]:
features = bag
target = df['Level']

In [63]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.3, random_state=RANDOM)

In [64]:
param_grid = {'penalty': ['l1','l2'],               
              'solver': ['lbfgs', 'newton-cg'],              
              'C': [1.0, 10.0, 100.0]}
              

model = LogisticRegression(random_state=RANDOM, n_jobs=1, multi_class='auto')

gs= GridSearchCV(model, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [65]:
gs.fit(features_train, target_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5, estimator=LogisticRegression(n_jobs=1, random_state=123),
             n_jobs=-1,
             param_grid={'C': [1.0, 10.0, 100.0], 'penalty': ['l1', 'l2'],
                         'solver': ['lbfgs', 'newton-cg']},
             scoring='accuracy', verbose=1)

In [66]:
print('Best parameter set: %s ' % gs.best_params_)
print('CV Accuracy: %.3f' % gs.best_score_)

Best parameter set: {'C': 100.0, 'penalty': 'l2', 'solver': 'lbfgs'} 
CV Accuracy: 0.696


In [67]:
best_model = gs.best_estimator_
print('Test Accuracy: %.3f' % best_model.score(features_test, target_test))

Test Accuracy: 0.695


In [68]:
predicted = best_model.predict(features_test)
df_test = pd.DataFrame()
df_test['Level_target'] = target_test
df_test = df_test.reset_index()
df_test['Level_pred'] = pd.Series(predicted)
df_test.sample(8)

,index,Level_target,Level_pred
41,157,B2,B2
79,218,C1,B2
51,251,B2,B2
37,258,B2,B1
53,236,B2,B2
14,29,B2,B2
17,239,B2,B2
42,72,B1,B2


Можем увидеть, что модель худо-бедно предсказывает уровень сложности текста.

Теперь надо собрать из этого нечто, чтоб любой входящий сырой текст довести до предсказания. 

In [69]:
import pickle

In [70]:
pkl_filename = "pickle_model.pkl" 
with open(pkl_filename, 'wb') as file: 
    pickle.dump(best_model, file)

In [ ]:
# Кусочек для приложения - нужно перенести сюда векторизацию - как-то составить словарь фичей.

  
def preproc_subs():

    stop = stopwords.words('english') + ["i'm"]

    del_n = re.compile('\n')                  # перенос каретки
    del_tags = re.compile('<[^>]*>')          # html-теги
    del_brackets = re.compile('\([^)]*\)')    # содержимое круглых скобок
    clean_text = re.compile('[^а-яa-z\s\']')  # все небуквенные символы кроме пробелов и апострофов
    del_spaces = re.compile('\s{2,}')         

    # Удаление символов
    def prepare_text(text):
        text = del_n.sub(' ', str(text).lower())
        text = del_tags.sub('', text)
        text = del_brackets.sub('', text)
        res_text = clean_text.sub('', text)
        res_text = res_text.lower()  #перевод в нижний регистр
        return del_spaces.sub(' ',res_text)
        
    # Удаление стоп-слов с вызовом функции с удалением символов
    def df_stop_words(Sr, stop):
        for itm in range(len(Sr)):
            line = Sr[itm]
            line = prepare_text(line)
            line = [w for w in line.split() if w not in stop]
            Sr[itm] = line
        return Sr    


    # Лемматизация, поиск корней с вызовом функции удаления стоп-слов   
    def lemmatise_srt(Sr, stop):
        df_stop_words(Sr, stop)
        stemmer = WordNetLemmatizer()
        for itm in range(len(Sr)):
            line = Sr[itm]
            line = [stemmer.lemmatize(w) for w in line]
            Sr[itm] =  str(line)
        return Sr
          

    path = 'Dir' # Путь к папке, в которую будет загружаться клиентский файл субтитров   

    df = pd.DataFrame(columns = ['Movie', 'Srt'])
    # Данные на вход - файл субтитров
    for file in os.listdir(path):
        infile = pysrt.open(os.path.join(path, file), encoding='iso-8859-1') # пользуемся этим при указании пути в папку сфайлом, а что с одиночным файлом?
        
        txt = infile.text # чтение субтитра
        df.loc[len(df)] = [file[:-4], txt]


    df['Srt'] = lemmatise_srt(df['Srt'], stop) 

    # Векторизация
    from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

    #count = CountVectorizer()
    bag = count.transform(df['Srt'])
    #tfidfconverter = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

    bag = tfidfconverter.transform(bag).toarray() # преобразует текст в цифры

    import pickle
    with open('pickle_model.pkl', 'rb') as file: 
        model = pickle.load(file)
        
    # Предсказание уровня языка по субтитрам фильма

    predicted = model.predict(bag)
    df['Level_pred'] = pd.Series(predicted)
    df.to_csv('answer.csv', index=False)




preproc_subs()

    


